#### Perform Analysis on the Dataset

- The user can perform analysis on the dataset.
- The types of data structures supported:
    - Tensors (With AutoDP + SMPC)
    - Numpy Arrays (Without AutoDP and SMPC)
- Two datasets from different domains can interact with each other (The operations are backed  by SMPC):
- The user can perform computation on the private data and access results via ADP.

List of all operations are being developed:
https://docs.google.com/spreadsheets/d/10GOdTuOxcaTJFENWAW8MAVDZYLEyJJJvdb0iUSt6Apw/edit#gid=2082830269

In [ ]:
# Let's login to a domain

# Let's assume we have the url to the domain node
# Let's login to the canada domain
ca_domain = sy.login(
    url="https://ca.openmined.org"
    email="sheldon@caltech.edu",
    password="bazinga",
    port=8081
)

# Let's login to the italy domain

it_domain = sy.login(
    url="https://it.openmined.org",
    email="sheldon@caltech.edu",
    password="bazinga",
    port=8082,
)

In [ ]:
# Let's select the trade dataset from both these domain
# Assumption: the dataset at index zero is the trade dataset and asset name is `feb-2020`

# Canada trade dataset
ca_trade_dataset = ca_domain.datasets[0]["feb-2020"]

# Italy trade dataset
it_trade_dataset = it_domain.datasets[0]["it-2020"]

In [ ]:
# Performing a private-private computation

# Sum the values in the columns
result = ca_trade_dataset + it_trade_dataset

In [ ]:
# If both ca_trade_dataset and it_trade_dataset have private Syft Tensors, then
published_result = result.publish(sigma=10)

In [16]:
# Get the results from the computation
# If the publish operation is still in process, and user tries to get results, then throw an error
published_result.get()


    ResultNotReadyException:
        The published results are not yet ready. Please try again after some time.



In [13]:
# If the publish operation failed for some reason and user tries to get result.
# Show the stack trace of the error in that case.
published_result.get()


    UnknownPrivateTensorException:
        The private tensor you're trying to access does not exists.



In [21]:
# If the publish operation is successful, then return the result.
published_result.get()

array([[153,  66, 102, 109],
       [ 90,  40,  93,  31],
       [ 49,  89,  43,  38],
       [ 82,   9,  78, 153]], dtype=int32)

In [28]:
# Now, let's assume that ca_trade_dataset is a PrivateTensor while it_trade_dataset is a normal TensorPointer
# i.e. AutoDP budgeting is applied to ca_trade_dataset but not to it_trade_dataset
# In such a case, `.publish` cannot be performed.
published_result = result.publish(sigma=10)


    ForbiddenOperationException:
        Publish operation cannot be performed, since AutoDP budgeting cannot be
        applied to one of the operands. Instead you can request results using `.request`
        method.



In [35]:
# Let's assume, the user is requesting results but, one of the data owners denies the results.
result_ptr = result.request(reason="Sum of the two columns")

# Let's say, Italy denied the request for result.
result_ptr.get()


    PermissionDenied:
        Your request to access the data has been denied by the `Italy Domain`.
        Please check `.requests` for more details on the request status.



In [ ]:
# If user has say epsilon of 3 and they published results that require epsilon of 5, then
# should we warn the user, or simple use burn the 3 epsilon and return the result.

In [ ]:
# Since, currently the values are restricted to `int32`, so inorder to perform an operation
# resulting in a float, we need to add some fix_precision to `int32` inorder to consider the fractional/decimal 
# values in our result.
# e.g. if the input value is: 5
# a fix_precision of 2 results in -> 500 (i.e. a float representation of the same value is 5.00)
# Now, the operation of 500/2, results in 250 (2.5 in float), thus preserving the decimal part of the result.

# Let's add the results
result = ca_trade_dataset + it_trade_dataset
# Perform a mean operation
result_mean = result.fix_precision().mean()  # default = 3, number of decimal places to preserve.

In [11]:
# If the fix precision is not implemented for any function, then throw a warning to the user
result_mean = result.fix_precision().mean()


    FixPrecisionNotImplemented:
        Fix precision is not implemented for the `division` function. The fractional part 
        of the result cannot be preserved.



In [14]:
# Let's assume, the `std` functionality of the numpy array is not implemented
# then throw an error

result = ca_trade_dataset + it_trade_dataset
stdeviation = np.std(result)


    NotImplementedError:
        The function `std` is currently not supported. We apologise for the inconvenience 🙏
        and will be adding support for more these methods very soon! 
        You can help us by creating a feature request for the desired functionality using the
        following link: https://github.com/OpenMined/PySyft/issues



#### Dummy Dataset

In [1]:
import pandas as pd
import numpy as np
from enum import Enum
import uuid
import datetime
import json


class bcolors(Enum):
    HEADER = "\033[95m"
    OKBLUE = "\033[94m"
    OKCYAN = "\033[96m"
    OKGREEN = "\033[92m"
    WARNING = "\033[93m"
    FAIL = "\033[91m"
    ENDC = "\033[0m"
    BOLD = "\033[1m"
    UNDERLINE = "\033[4m"

In [15]:
published_results_in_process=f"""
    {bcolors.FAIL.value}ResultNotReadyException:{bcolors.ENDC.value}
        The published results are not yet ready. Please try again after some time.
"""
print(published_results_in_process)


    ResultNotReadyException:
        The published results are not yet ready. Please try again after some time.



In [2]:
published_results_failed=f"""
    {bcolors.FAIL.value}UnknownPrivateTensorException:{bcolors.ENDC.value}
        The private tensor you're trying to access does not exists.

        {bcolors.BOLD.value}StackTrace
        -----------{bcolors.ENDC.value}
        ..
        ..
        ..
"""
published_results_failed_v2=f"""
    {bcolors.FAIL.value}UnknownPrivateTensorException:{bcolors.ENDC.value}
        The private tensor you're trying to access does not exists.
"""
print(published_results_failed)


    UnknownPrivateTensorException:
        The private tensor you're trying to access does not exists.

        StackTrace
        -----------
        ..
        ..
        ..



In [4]:
published_result = np.random.randint(1, 256, (4, 4)).astype(np.int32)

In [5]:
private_budget_not_applied = f"""
    {bcolors.FAIL.value}ForbiddenOperationException:{bcolors.ENDC.value}
        Publish operation cannot be performed, since AutoDP budgeting cannot be
        applied to one of the operands. Instead you can request results using {bcolors.BOLD.value}`.request`{bcolors.ENDC.value}
        method.
"""
print(private_budget_not_applied)


    ForbiddenOperationException:
        Publish operation cannot be performed, since AutoDP budgeting cannot be
        applied to one of the operands. Instead you can request results using `.request`
        method.



In [6]:
request_denied_by_a_do = f"""
    {bcolors.FAIL.value}PermissionDenied:{bcolors.ENDC.value}
        Your request to access the data has been denied by the {bcolors.BOLD.value}`Italy Domain`{bcolors.ENDC.value}.
        Please check {bcolors.BOLD.value}`.requests`{bcolors.ENDC.value} for more details on the request status.
"""

print(request_denied_by_a_do)


    PermissionDenied:
        Your request to access the data has been denied by the `Italy Domain`.
        Please check `.requests` for more details on the request status.



In [10]:
fix_precision_not_implemented = f"""
    {bcolors.WARNING.value}FixPrecisionNotImplemented:{bcolors.ENDC.value}
        Fix precision is not implemented for the `division` function. The fractional part 
        of the result cannot be preserved.
"""
print(fix_precision_not_implemented)


    FixPrecisionNotImplemented:
        Fix precision is not implemented for the `division` function. The fractional part 
        of the result cannot be preserved.



In [11]:
function_not_implemented = f"""
    {bcolors.FAIL.value}NotImplementedError:{bcolors.ENDC.value}
        The function `std` is currently not supported. We apologise for the inconvenience 🙏
        and will be adding support for more these methods very soon! 
        You can help us by creating a feature request for the desired functionality using the
        following link: https://github.com/OpenMined/PySyft/issues
"""

In [12]:
print(function_not_implemented)


    NotImplementedError:
        The function `std` is currently not supported. We apologise for the inconvenience 🙏
        and will be adding support for more these methods very soon! 
        You can help us by creating a feature request for the desired functionality using the
        following link: https://github.com/OpenMined/PySyft/issues

